# Working with Events

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database, file store, and transcript usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Get all votes for a single person

Query and merge the `person` and `vote` tables.

In [2]:
# Filtering not supported for open access API yet, get all people then dataframe filter to target
people = pd.DataFrame(db.select_rows_as_list("person"))
teresa = people.loc[people["full_name"] == "Teresa Mosqueda"]

# Pull voting data
votes = pd.DataFrame(db.select_rows_as_list("vote"))

# Merge data
teresa_voting_record = teresa.merge(
    votes,
    left_on="person_id",
    right_on="person_id",
    suffixes=("_person", "_vote")
)

teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision,event_minutes_item_id,legistar_event_item_vote_id,vote_id
0,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 19:45:56.230406,In Favor,ee8c1a1e-6264-42b4-a468-e96fa05b6431,46449,00851688-5f49-407c-82f6-a3535bcbc853
1,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 20:06:43.982793,In Favor,946c0537-8700-4755-ae26-22df2fd98022,46554,054c3870-aac4-49bd-9bcc-6f290b4d5e15
2,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 20:06:32.471476,In Favor,0bc51be9-c294-48a6-8d5c-90d0fa0a8b9b,46530,0d0c38e6-a260-4b12-be2b-fd8674c69d05
3,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 20:06:24.656702,In Favor,95eb3afc-d5be-4303-9370-a236cd3a9a29,46590,11ffb3c2-734f-4b1b-a184-0ec87b7b2c18
4,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 19:46:11.926495,In Favor,0346d121-6fb4-419e-9398-f2df5ffbdeb5,46489,19139c93-2ef1-47ae-bfd9-c626282bfdfe


This is great, but it's missing context of what was actually being voted on. Currently we can see that she voted on instances of `event_minutes_item`, but what were those. Follow the same procedure of `select_rows_as_list` -> `merge` to join the data of the other tables to Teresa's voting record.

In [3]:
# Get event minutes item context data
event_minutes_items = pd.DataFrame(db.select_rows_as_list("event_minutes_item"))
minutes_items = pd.DataFrame(db.select_rows_as_list("minutes_item"))
events = pd.DataFrame(db.select_rows_as_list("event"))

# Merge minutes item data
event_minutes_items = event_minutes_items.merge(
    minutes_items,
    left_on="minutes_item_id",
    right_on="minutes_item_id",
    suffixes=("_event_minutes_item", "_minutes_item")
)

# Merge event data
event_minutes_items = event_minutes_items.merge(
    events,
    left_on="event_id",
    right_on="event_id",
    suffixes=("_event_minutes_item", "_event")
)

# Merge above event minutes item data with Teresa's voting record
teresa_voting_record = teresa_voting_record.merge(
    event_minutes_items,
    left_on="event_minutes_item_id",
    right_on="event_minutes_item_id",
    suffixes=("_voting_record", "_event_minutes_item")
)

# Sort by event datetime
teresa_voting_record = teresa_voting_record.sort_values(by=["event_datetime"])
teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision_voting_record,event_minutes_item_id,...,name,agenda_file_uri,body_id,created,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
3,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 22:48:46.523376,In Favor,d22175c9-99e8-4d82-a2be-5ba09a6992db,...,CB 119553,http://legistar2.granicus.com/seattle/meetings...,ef506236-3c85-4393-b8fb-529f21957e44,2019-07-20 20:06:17.809705,2019-07-01 14:00:00,4021,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
4,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 22:48:39.540836,In Favor,9fd4be81-551c-465b-a04d-8da6a3364f7e,...,CB 119559,http://legistar2.granicus.com/seattle/meetings...,ef506236-3c85-4393-b8fb-529f21957e44,2019-07-20 20:06:17.809705,2019-07-01 14:00:00,4021,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
5,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 22:49:04.211789,In Favor,cc7d2c1e-f02d-420f-8650-dfb5a7c8bf30,...,CB 119547,http://legistar2.granicus.com/seattle/meetings...,ef506236-3c85-4393-b8fb-529f21957e44,2019-07-20 20:06:17.809705,2019-07-01 14:00:00,4021,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
6,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 22:48:57.828960,In Favor,2bfe6638-6b66-4fca-a12a-88b1598b7215,...,CB 119552,http://legistar2.granicus.com/seattle/meetings...,ef506236-3c85-4393-b8fb-529f21957e44,2019-07-20 20:06:17.809705,2019-07-01 14:00:00,4021,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
2,2019-07-20 18:36:09.978150,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,4ce73847-710f-4812-ad85-a3ff2cb43e26,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-20 22:48:54.896875,In Favor,ca0edf17-1f94-40f0-a7e1-08b25bd76840,...,"Appointment of Victor M. Lozada as member, Dom...",http://legistar2.granicus.com/seattle/meetings...,11ec4e30-c8f1-49ce-b39d-14126991cc42,2019-07-20 19:02:32.603228,2019-07-11 09:30:00,3994,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ho...


In [4]:
teresa_voting_record.loc[0]

created_person                                        2019-07-20 18:36:09.978150
email                                                Teresa.Mosqueda@seattle.gov
full_name                                                        Teresa Mosqueda
legistar_person_id                                                           594
person_id                                   4ce73847-710f-4812-ad85-a3ff2cb43e26
phone                                                               206-684-8806
website                                  http://www.seattle.gov/council/mosqueda
created_vote                                          2019-07-20 22:48:35.774484
decision_voting_record                                                  In Favor
event_minutes_item_id                       2aab4483-8ae0-4e90-bd8c-5fab9ad2bbde
legistar_event_item_vote_id                                                46709
vote_id                                     1a80465c-2193-44ca-b5d9-365417ceeb80
created_event_minutes_item  